In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Import librarys
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#pandas dataframe config
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format

In [3]:
DATA_PATH = '../input/store-sales-time-series-forecasting/'

In [4]:
for f in os.listdir(DATA_PATH):
    print(f)

In [5]:
# At first instance, we import every datasource as a dataframe 
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_stores = pd.read_csv(os.path.join(DATA_PATH, 'stores.csv'))
# For the transactions dataset, we sort them by store number and date, to help us visualizing the data chronologically
df_transactions = pd.read_csv(os.path.join(DATA_PATH, 'transactions.csv')).sort_values(['store_nbr', 'date'])
df_oil = pd.read_csv(os.path.join(DATA_PATH, 'oil.csv'))
df_holidays_events = pd.read_csv(os.path.join(DATA_PATH, 'holidays_events.csv'))

In [6]:
#For the column date of each dataframe that contains it, we convert te data type to pandas datetime, as the original datetime format is not supported by pd
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

In [7]:
sns.heatmap(data=df_train.corr(), vmin=-1, vmax=1, cmap = 'RdBu', annot=True, square = True)

In [8]:
# We duplicate the train dataframe in order to process it in a different memory address
process_train = df_train.copy()

In [9]:
# We ensure that the column date has a supported date format. This is just preventive, as we had already converted the datatype of df_train prior to creating this copy
process_train['date'] = pd.to_datetime(process_train['date'])
#We set the date as the index of the dataframe
process_train = process_train.set_index('date')
#We drop the column containing the id numbers, as now our id's will be the dates
process_train = process_train.drop('id',axis = 1)
# We indicate that the columns store number and family will be treated as category information. This helps optimizing the performance
process_train[['store_nbr','family']].astype('category') #reduces ram usage
process_train

In [10]:
# We group the information by family of products, and resample it to a monthly scale
month_family = process_train.groupby('family').resample('M').sales.sum() #resample to monthly sales 

fig, ax = plt.subplots(figsize=(7,7))
plt.barh(month_family.groupby('family').sum().sort_values().index,month_family.groupby('family').sum().sort_values())
ax.set(title='Total Sales by Product Family')
plt.show()

Transactions

In [11]:
# We create a temporal dataframe were we sum the sales of each store every day
df_train_temp = df_train.groupby(['date', 'store_nbr']).sales.sum().reset_index()

In [12]:
df_train_temp

In [13]:
#Now, we meerge the previous dataframe with our transactions information.
df_aux_merge = pd.merge(df_train_temp, df_transactions, how = 'left')

In [14]:
sns.heatmap(data=df_aux_merge.corr(), vmin=-1, vmax=1, cmap = 'RdBu', annot=True, square = True)

In [15]:
aux_a = df_transactions.copy()
aux_a['year'] = aux_a.date.dt.year
aux_a['month'] = aux_a.date.dt.month

In [16]:
sns.heatmap(data=aux_a.corr(), vmin=-1, vmax=1, cmap = 'RdBu', annot=True, square = True)

In [17]:
pd.date_range(start = '2013-01-01', end = '2017-08-15' ).difference(df_oil.index)

In [18]:
# We prepare out oil dataset by setting a correct format for date and setting date as the index
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil = df_oil.set_index('date')

In [19]:
df_oil = df_oil.resample('1D').sum()
df_oil.reset_index()

In [20]:
pd.date_range(start = '2013-01-01', end = '2017-08-15' ).difference(df_oil.index)

In [21]:
# Here we replace all NaN values for 0, as we will use this column to compute the interpolated price
df_oil['dcoilwtico'] = np.where(df_oil['dcoilwtico']==0, np.nan, df_oil['dcoilwtico'])
df_oil['interpolated_price'] = df_oil.dcoilwtico.interpolate()

In [22]:
# We drop the column now that we obtained the interpolated price
df_oil = df_oil.drop('dcoilwtico',axis=1)


In [23]:
df_oil['price_chg'] = df_oil.interpolated_price - df_oil.interpolated_price.shift(1)
df_oil['pct_chg'] = df_oil['price_chg']/df_oil.interpolated_price.shift(-1)

In [24]:
fig,ax = plt.subplots(figsize=(15,5))
plt.plot(df_oil['interpolated_price'])
plt.title('Oil Price over time')

plt.show()

In [25]:
# We make sure that the dateitme format is correct on this dataframe
df_train['date'] = pd.to_datetime(df_train['date'])
# We group it by date and we sum the values of sales for each day.
df_dates = df_train.groupby(df_train.date)['sales'].sum().reset_index()

In [26]:
df_dates_t = df_transactions.groupby(df_transactions.date)['transactions'].sum().reset_index()

In [27]:
df_dates.plot(kind = 'scatter', x = 'date', y = 'sales')

In [28]:
df_dates.sort_values(by=['sales'])

In [29]:
daily_total_sales = df_dates.copy()

In [30]:
daily_total_sales = daily_total_sales.set_index(pd.to_datetime(daily_total_sales['date']))


In [31]:
daily_total_sales = daily_total_sales.resample('1D').sum()
daily_total_sales

In [32]:
df_oil.interpolated_price.loc['2013-01-01':'2017-08-15']


In [33]:
plt.scatter(daily_total_sales,df_oil.interpolated_price.loc['2013-01-01':'2017-08-15'],alpha=0.2)
plt.ylabel('oil price')
plt.xlabel('daily total sales')
plt.show()

# Limpieza de datos

**Eliminando columnas y datos innecesarios**

Eliminar id

In [34]:
df_train_clean = df_train.copy()
df_test_clean = df_test.copy()

df_train_clean = df_train_clean.drop(["id"], axis=1)
df_test_clean = df_test_clean.drop(["id"], axis=1)

Se observó que los dias primero son dias distopicos del año en donde las ventas son minimas

In [35]:
df_fd=df_train_clean.loc[(df_train_clean["date"].dt.day == 1) & (df_train_clean["date"].dt.month == 1)].index
df_train_clean=df_train_clean.drop(df_fd)
df_fd=df_test_clean.loc[(df_test_clean["date"].dt.day == 1) & (df_test_clean["date"].dt.month == 1)].index
df_test_clean=df_test_clean.drop(df_fd)


In [36]:
df_dates.plot(kind = 'scatter', x = 'date', y = 'sales')

Considerando que hubo un terremoto de gran magnitud que impacta en las observaciones del dataset, decidimos eliminar este mes. Esto fue porque el terremoto es una anomalía en los datos que no se repite con frecuencia, por lo que termina entorpeciendo el análisis de datos al introducir un comportamiento anómalo en los mismos.

In [37]:
df_fd=df_train_clean.loc[(df_train_clean["date"].dt.month == 4) & (df_train_clean["date"].dt.day >= 16) & (df_train_clean["date"].dt.day <= 31) & (df_train_clean["date"].dt.year == 2016)].index
df_train_clean=df_train_clean.drop(df_fd)

df_fd=df_train_clean.loc[(df_train_clean["date"].dt.month == 5) & (df_train_clean["date"].dt.day >= 1) & (df_train_clean["date"].dt.day <= 16) & (df_train_clean["date"].dt.year == 2016)].index
df_train_clean=df_train_clean.drop(df_fd)

df_fd=df_test_clean.loc[(df_test_clean["date"].dt.month == 4) & (df_test_clean["date"].dt.day >= 16) & (df_test_clean["date"].dt.day <= 31) & (df_test_clean["date"].dt.year == 2016)].index
df_test_clean=df_test_clean.drop(df_fd)

df_fd=df_test_clean.loc[(df_test_clean["date"].dt.month == 5) & (df_test_clean["date"].dt.day >= 1) & (df_test_clean["date"].dt.day <= 16) & (df_test_clean["date"].dt.year == 2016)].index
df_test_clean=df_test_clean.drop(df_fd)

In [38]:
df_dates = df_train_clean.groupby(df_train_clean.date)['sales'].sum().reset_index()


In [39]:
df_dates.plot(kind = 'scatter', x = 'date', y = 'sales')

Los datos ya se encuentran limpios y libres de anomalias

In [40]:
df_dates

Ahora se procederá a crear el DataFrame en el que se trabajará

In [41]:
df_mrg = pd.merge(df_dates, df_dates_t, on="date")

Eliminar columnas que no se ocupan

In [42]:
df_mrg

In [43]:
df_holidays_events

In [44]:
df_he = df_holidays_events.drop(["locale_name", "description", "transferred", "type"], axis=1)
df_he

In [45]:
df_he["locale"].value_counts()

In [46]:
df_di=df_he.loc[(df_he["locale"] == 'Regional')].index
df_he=df_he.drop(df_di)

In [47]:
df_he = df_he.replace(to_replace="Local",
           value=1)
df_he = df_he.replace(to_replace="National",
           value=1)

In [48]:
df_he.columns = df_he.columns.str.replace('locale', 'IsHoliday')
df_he


In [49]:
df_he["date"] = pd.to_datetime(df_he['date'])

In [50]:
df_he

In [51]:
df_mrg

In [52]:
df_mrg['IsHoliday'] = df_he['IsHoliday']

In [53]:
df_mrg['IsHoliday'] = df_mrg['IsHoliday'].fillna(0)

In [54]:
df_mrg

In [55]:
df_mrg.corr()